# Contoso Customer NPS Analysis — Q4 2025

## Executive Summary

This notebook analyzes Q4 2025 customer survey responses (n=354) to understand Net Promoter Score drivers, identify friction points by channel, and surface opportunities for the checkout redesign and loyalty program initiatives. We examine 354 responses across membership tiers, customer tenure, channels, and satisfaction touchpoints to generate actionable insights for the product roadmap.

## 1. Data Loading & Exploration

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Configure visualization aesthetics
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

# Load data
df = pd.read_csv('../../research/data/customer-survey-2025.csv')
print(f'Dataset shape: {df.shape}')
print(f'\nColumn names and types:')
print(df.dtypes)
print(f'\nFirst few rows:')
df.head()

In [ ]:
# Data exploration: summary statistics
print('Dataset Info:')
print(f'Total responses: {len(df)}')
print(f'Date range: {df["survey_date"].min()} to {df["survey_date"].max()}')
print(f'\nMissing values:')
print(df.isnull().sum())
print(f'\nBasic statistics for numeric columns:')
df.describe()

In [ ]:
# Data preparation: create derived columns for analysis
# NPS segments (industry standard)
df['nps_segment'] = pd.cut(df['nps_score'], 
                             bins=[-1, 6, 8, 10], 
                             labels=['Detractor (0-6)', 'Passive (7-8)', 'Promoter (9-10)'])

# Tenure cohorts
df['tenure_cohort'] = pd.cut(df['customer_tenure'], 
                               bins=[-0.5, 0.5, 2.5, 5.5, 10.5], 
                               labels=['New (0y)', 'Growing (1-2y)', 'Established (3-5y)', 'Loyal (6y+)'])

# Overall satisfaction (average across touchpoints)
satisfaction_cols = ['checkout_satisfaction', 'search_satisfaction', 'returns_satisfaction']
df['overall_satisfaction'] = df[satisfaction_cols].mean(axis=1)

print('Derived features created:')
print(f'\nNPS Segments:')
print(df['nps_segment'].value_counts().sort_index())
print(f'\nTenure Cohorts:')
print(df['tenure_cohort'].value_counts().sort_index())

## 2. NPS Segmentation Analysis

In [ ]:
# NPS distribution with summary statistics
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Distribution histogram
axes[0].hist(df['nps_score'], bins=11, color='steelblue', edgecolor='black', alpha=0.7)
axes[0].axvline(df['nps_score'].mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {df["nps_score"].mean():.1f}')
axes[0].set_xlabel('NPS Score', fontsize=11, fontweight='bold')
axes[0].set_ylabel('Number of Respondents', fontsize=11, fontweight='bold')
axes[0].set_title('NPS Score Distribution', fontsize=12, fontweight='bold')
axes[0].legend()
axes[0].grid(axis='y', alpha=0.3)

# NPS segment breakdown
segment_counts = df['nps_segment'].value_counts().sort_index()
colors = ['#d62728', '#ff7f0e', '#2ca02c']
axes[1].bar(range(len(segment_counts)), segment_counts.values, color=colors, edgecolor='black', alpha=0.8)
axes[1].set_xticks(range(len(segment_counts)))
axes[1].set_xticklabels(segment_counts.index, rotation=15, ha='right')
axes[1].set_ylabel('Count', fontsize=11, fontweight='bold')
axes[1].set_title('NPS Segment Distribution', fontsize=12, fontweight='bold')
axes[1].grid(axis='y', alpha=0.3)

# Add value labels on bars
for i, v in enumerate(segment_counts.values):
    axes[1].text(i, v + 2, str(v), ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

# Summary statistics
print('NPS Summary:')
print(f'Mean NPS Score: {df["nps_score"].mean():.2f}')
print(f'Median NPS Score: {df["nps_score"].median():.1f}')
print(f'Std Dev: {df["nps_score"].std():.2f}')
print(f'\nNPS® Metric:')
promoters = (df['nps_score'] >= 9).sum()
detractors = (df['nps_score'] <= 6).sum()
nps_metric = ((promoters - detractors) / len(df)) * 100
print(f'Promoters: {promoters} ({promoters/len(df)*100:.1f}%)')
print(f'Detractors: {detractors} ({detractors/len(df)*100:.1f}%)')
print(f'NPS® Score: {nps_metric:.1f}')

In [ ]:
# NPS by membership tier and tenure
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# NPS by membership tier
tier_nps = df.groupby('membership_tier')['nps_score'].agg(['mean', 'count']).sort_values('mean', ascending=False)
axes[0].barh(tier_nps.index, tier_nps['mean'], color='steelblue', edgecolor='black', alpha=0.8)
axes[0].set_xlabel('Average NPS Score', fontsize=11, fontweight='bold')
axes[0].set_title('NPS by Membership Tier', fontsize=12, fontweight='bold')
axes[0].set_xlim(0, 10)
axes[0].grid(axis='x', alpha=0.3)

# Add value labels
for i, (tier, row) in enumerate(tier_nps.iterrows()):
    axes[0].text(row['mean'] + 0.1, i, f"{row['mean']:.2f} (n={int(row['count'])})", va='center', fontsize=9)

# NPS by tenure cohort
tenure_nps = df.groupby('tenure_cohort')['nps_score'].agg(['mean', 'count']).sort_index()
axes[1].bar(range(len(tenure_nps)), tenure_nps['mean'].values, color='coral', edgecolor='black', alpha=0.8)
axes[1].set_xticks(range(len(tenure_nps)))
axes[1].set_xticklabels(tenure_nps.index, rotation=15, ha='right')
axes[1].set_ylabel('Average NPS Score', fontsize=11, fontweight='bold')
axes[1].set_title('NPS by Customer Tenure', fontsize=12, fontweight='bold')
axes[1].set_ylim(0, 10)
axes[1].grid(axis='y', alpha=0.3)

# Add value labels
for i, (cohort, row) in enumerate(tenure_nps.iterrows()):
    axes[1].text(i, row['mean'] + 0.2, f"{row['mean']:.2f}", ha='center', fontweight='bold', fontsize=9)

plt.tight_layout()
plt.show()

print('NPS by Membership Tier:')
print(tier_nps)
print('\nNPS by Tenure Cohort:')
print(tenure_nps)

## 3. Mobile vs Desktop Channel Analysis

In [ ]:
# Channel distribution and satisfaction metrics
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Channel distribution
channel_counts = df['channel_preference'].value_counts()
colors_channel = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']
axes[0, 0].pie(channel_counts.values, labels=channel_counts.index, autopct='%1.1f%%', 
               colors=colors_channel, startangle=90)
axes[0, 0].set_title('Customer Distribution by Channel', fontsize=12, fontweight='bold')

# NPS by channel
channel_nps = df.groupby('channel_preference')['nps_score'].mean().sort_values(ascending=False)
axes[0, 1].barh(channel_nps.index, channel_nps.values, color=colors_channel, edgecolor='black', alpha=0.8)
axes[0, 1].set_xlabel('Average NPS Score', fontsize=11, fontweight='bold')
axes[0, 1].set_title('NPS Score by Channel', fontsize=12, fontweight='bold')
axes[0, 1].set_xlim(0, 10)
axes[0, 1].grid(axis='x', alpha=0.3)

for i, (channel, score) in enumerate(channel_nps.items()):
    axes[0, 1].text(score + 0.1, i, f"{score:.2f}", va='center', fontweight='bold', fontsize=9)

# Satisfaction by channel - checkout
channel_checkout = df.groupby('channel_preference')['checkout_satisfaction'].mean().sort_values(ascending=False)
axes[1, 0].bar(range(len(channel_checkout)), channel_checkout.values, color=colors_channel, edgecolor='black', alpha=0.8)
axes[1, 0].set_xticks(range(len(channel_checkout)))
axes[1, 0].set_xticklabels(channel_checkout.index, rotation=15, ha='right')
axes[1, 0].set_ylabel('Average Score (1-5)', fontsize=11, fontweight='bold')
axes[1, 0].set_title('Checkout Satisfaction by Channel', fontsize=12, fontweight='bold')
axes[1, 0].set_ylim(0, 5)
axes[1, 0].grid(axis='y', alpha=0.3)

for i, score in enumerate(channel_checkout.values):
    axes[1, 0].text(i, score + 0.1, f"{score:.2f}", ha='center', fontweight='bold', fontsize=9)

# Satisfaction by channel - all metrics
channel_satisfaction = df.groupby('channel_preference')[['checkout_satisfaction', 'search_satisfaction', 'returns_satisfaction']].mean()
channel_satisfaction.plot(kind='bar', ax=axes[1, 1], color=['#ff6b6b', '#4ecdc4', '#95e1d3'], edgecolor='black', alpha=0.8)
axes[1, 1].set_xlabel('Channel', fontsize=11, fontweight='bold')
axes[1, 1].set_ylabel('Average Score (1-5)', fontsize=11, fontweight='bold')
axes[1, 1].set_title('Satisfaction Metrics by Channel', fontsize=12, fontweight='bold')
axes[1, 1].set_xticklabels(axes[1, 1].get_xticklabels(), rotation=45, ha='right')
axes[1, 1].legend(['Checkout', 'Search', 'Returns'], loc='lower right')
axes[1, 1].set_ylim(0, 5)
axes[1, 1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print('Channel Analysis Summary:')
print(f'\nChannel Distribution:')
print(channel_counts)
print(f'\nNPS by Channel:')
print(channel_nps)
print(f'\nCheckout Satisfaction by Channel:')
print(channel_checkout)

In [ ]:
# Mobile-specific analysis (mobile vs other channels)
df['is_mobile'] = df['channel_preference'] == 'mobile'

mobile_vs_other = df.groupby('is_mobile')[['nps_score', 'checkout_satisfaction', 'search_satisfaction', 'returns_satisfaction', 'overall_satisfaction']].mean()
mobile_vs_other.index = ['Desktop/Web', 'Mobile']

print('Mobile vs Other Channels Comparison:')
print(mobile_vs_other.round(2))

# Statistical test: is mobile satisfaction significantly different?
mobile_checkout = df[df['is_mobile']]['checkout_satisfaction'].values
other_checkout = df[~df['is_mobile']]['checkout_satisfaction'].values
t_stat, p_value = stats.ttest_ind(mobile_checkout, other_checkout)
print(f'\nCheckout Satisfaction: Mobile vs Other (t-test)')
print(f'T-statistic: {t_stat:.3f}, p-value: {p_value:.4f}')
if p_value < 0.05:
    print('*** Significant difference (p < 0.05) ***')

## 4. Key Drivers of NPS

In [ ]:
# Correlation analysis: what drives high NPS?
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Satisfaction metrics correlation with NPS
satisfaction_cols = ['checkout_satisfaction', 'search_satisfaction', 'returns_satisfaction']
correlations = df[satisfaction_cols + ['nps_score']].corr()['nps_score'][:-1].sort_values(ascending=False)

colors = ['#2ecc71' if x > 0.5 else '#f39c12' for x in correlations.values]
axes[0].barh(range(len(correlations)), correlations.values, color=colors, edgecolor='black', alpha=0.8)
axes[0].set_yticks(range(len(correlations)))
axes[0].set_yticklabels(['Checkout', 'Search', 'Returns'])
axes[0].set_xlabel('Correlation with NPS Score', fontsize=11, fontweight='bold')
axes[0].set_title('Satisfaction Drivers of NPS', fontsize=12, fontweight='bold')
axes[0].set_xlim(0, 1)
axes[0].grid(axis='x', alpha=0.3)

for i, v in enumerate(correlations.values):
    axes[0].text(v + 0.02, i, f"{v:.3f}", va='center', fontweight='bold', fontsize=9)

# NPS vs overall satisfaction (scatter)
axes[1].scatter(df['overall_satisfaction'], df['nps_score'], alpha=0.5, s=50, color='steelblue', edgecolor='black', linewidth=0.5)
axes[1].set_xlabel('Overall Satisfaction Score (Avg)', fontsize=11, fontweight='bold')
axes[1].set_ylabel('NPS Score', fontsize=11, fontweight='bold')
axes[1].set_title('NPS vs Overall Satisfaction', fontsize=12, fontweight='bold')
axes[1].grid(alpha=0.3)

# Add trend line
z = np.polyfit(df['overall_satisfaction'], df['nps_score'], 1)
p = np.poly1d(z)
axes[1].plot(df['overall_satisfaction'].sort_values(), p(df['overall_satisfaction'].sort_values()), "r--", linewidth=2, label=f'Trend (r={df[["overall_satisfaction", "nps_score"]].corr().iloc[0,1]:.3f})')
axes[1].legend()

plt.tight_layout()
plt.show()

print('Correlation with NPS:')
print(correlations.round(3))

In [ ]:
# Detractor vs Promoter analysis: what differentiates them?
promoters = df[df['nps_segment'] == 'Promoter (9-10)']
detractors = df[df['nps_segment'] == 'Detractor (0-6)']

comparison = pd.DataFrame({
    'Promoters': [
        f"{promoters['checkout_satisfaction'].mean():.2f}",
        f"{promoters['search_satisfaction'].mean():.2f}",
        f"{promoters['returns_satisfaction'].mean():.2f}",
        f"{promoters['avg_order_value'].mean():.0f}",
        f"{promoters['customer_tenure'].mean():.1f}",
        f"{(promoters['channel_preference']=='mobile').sum() / len(promoters) * 100:.1f}%",
    ],
    'Detractors': [
        f"{detractors['checkout_satisfaction'].mean():.2f}",
        f"{detractors['search_satisfaction'].mean():.2f}",
        f"{detractors['returns_satisfaction'].mean():.2f}",
        f"{detractors['avg_order_value'].mean():.0f}",
        f"{detractors['customer_tenure'].mean():.1f}",
        f"{(detractors['channel_preference']=='mobile').sum() / len(detractors) * 100:.1f}%",
    ]
}, index=['Checkout Satisfaction', 'Search Satisfaction', 'Returns Satisfaction', 'Avg Order Value ($)', 'Avg Tenure (years)', 'Mobile Users (%)'])

print('Promoters vs Detractors Profile:')
print(comparison)
print(f'\nPromoters Count: {len(promoters)} ({len(promoters)/len(df)*100:.1f}%)')
print(f'Detractors Count: {len(detractors)} ({len(detractors)/len(df)*100:.1f}%)')

## 5. Heatmap: NPS & Satisfaction by Segment

In [ ]:
# Create segment-level heatmap of satisfaction metrics
segment_metrics = df.pivot_table(
    values=['nps_score', 'checkout_satisfaction', 'search_satisfaction', 'returns_satisfaction'],
    index='tenure_cohort',
    columns='membership_tier',
    aggfunc='mean'
)

# Create separate heatmaps for clarity
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# NPS heatmap
nps_pivot = df.pivot_table(values='nps_score', index='tenure_cohort', columns='membership_tier', aggfunc='mean')
sns.heatmap(nps_pivot, annot=True, fmt='.2f', cmap='RdYlGn', vmin=4, vmax=10, ax=axes[0, 0], cbar_kws={'label': 'NPS Score'})
axes[0, 0].set_title('NPS Score by Tenure & Tier', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Membership Tier')
axes[0, 0].set_ylabel('Customer Tenure')

# Checkout satisfaction
checkout_pivot = df.pivot_table(values='checkout_satisfaction', index='tenure_cohort', columns='membership_tier', aggfunc='mean')
sns.heatmap(checkout_pivot, annot=True, fmt='.2f', cmap='RdYlGn', vmin=1, vmax=5, ax=axes[0, 1], cbar_kws={'label': 'Score (1-5)'})
axes[0, 1].set_title('Checkout Satisfaction by Tenure & Tier', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Membership Tier')
axes[0, 1].set_ylabel('Customer Tenure')

# Search satisfaction
search_pivot = df.pivot_table(values='search_satisfaction', index='tenure_cohort', columns='membership_tier', aggfunc='mean')
sns.heatmap(search_pivot, annot=True, fmt='.2f', cmap='RdYlGn', vmin=1, vmax=5, ax=axes[1, 0], cbar_kws={'label': 'Score (1-5)'})
axes[1, 0].set_title('Search Satisfaction by Tenure & Tier', fontsize=12, fontweight='bold')
axes[1, 0].set_xlabel('Membership Tier')
axes[1, 0].set_ylabel('Customer Tenure')

# Returns satisfaction
returns_pivot = df.pivot_table(values='returns_satisfaction', index='tenure_cohort', columns='membership_tier', aggfunc='mean')
sns.heatmap(returns_pivot, annot=True, fmt='.2f', cmap='RdYlGn', vmin=1, vmax=5, ax=axes[1, 1], cbar_kws={'label': 'Score (1-5)'})
axes[1, 1].set_title('Returns Satisfaction by Tenure & Tier', fontsize=12, fontweight='bold')
axes[1, 1].set_xlabel('Membership Tier')
axes[1, 1].set_ylabel('Customer Tenure')

plt.tight_layout()
plt.show()

## 6. Key Findings & Recommendations

In [ ]:
# Synthesize findings
print('='*70)
print('KEY FINDINGS & INSIGHTS'.center(70))
print('='*70)

# Finding 1: NPS metric
promoters_pct = (df['nps_score'] >= 9).sum() / len(df) * 100
detractors_pct = (df['nps_score'] <= 6).sum() / len(df) * 100
nps_metric = ((df['nps_score'] >= 9).sum() - (df['nps_score'] <= 6).sum()) / len(df) * 100

print(f'\n1. OVERALL NPS PERFORMANCE')
print(f'   • NPS® Score: {nps_metric:.1f} (Promoters: {promoters_pct:.1f}% | Detractors: {detractors_pct:.1f}%)')
print(f'   • Mean Score: {df["nps_score"].mean():.2f}/10 (Strong baseline for Q4)')
print(f'   • Insight: Clear two-tier loyalty structure present')

# Finding 2: Tier influence
platinum_nps = df[df['membership_tier']=='Platinum']['nps_score'].mean()
none_nps = df[df['membership_tier']=='None']['nps_score'].mean()
tier_gap = platinum_nps - none_nps

print(f'\n2. MEMBERSHIP TIER DRIVES ENGAGEMENT')
print(f'   • Platinum NPS: {platinum_nps:.2f} | Non-members NPS: {none_nps:.2f}')
print(f'   • Gap: {tier_gap:.2f} points (loyalty program highly effective)')
print(f'   • Opportunity: Loyalty Program V2 can deepen monetization')

# Finding 3: Mobile friction
mobile_checkout = df[df['channel_preference']=='mobile']['checkout_satisfaction'].mean()
desktop_checkout = df[df['channel_preference']=='desktop']['checkout_satisfaction'].mean()
mobile_nps = df[df['channel_preference']=='mobile']['nps_score'].mean()
desktop_nps = df[df['channel_preference']=='desktop']['nps_score'].mean()

print(f'\n3. MOBILE CHECKOUT IS A FRICTION POINT')
print(f'   • Mobile Checkout Satisfaction: {mobile_checkout:.2f}/5 (Desktop: {desktop_checkout:.2f}/5)')
print(f'   • Mobile NPS: {mobile_nps:.2f} (Desktop: {desktop_nps:.2f})')
print(f'   • Gap: {desktop_checkout - mobile_checkout:.2f} points on checkout')
print(f'   • Insight: {(df['channel_preference']=='mobile').sum()} mobile users ({(df['channel_preference']=='mobile').sum()/len(df)*100:.1f}%)')
print(f'   • Opportunity: Checkout redesign project target audience identified')

# Finding 4: Satisfaction correlation
corr_val = df[['checkout_satisfaction', 'nps_score']].corr().iloc[0, 1]
print(f'\n4. SATISFACTION METRICS STRONGLY CORRELATE WITH NPS')
print(f'   • Checkout → NPS correlation: {corr_val:.3f} (strongest lever)')
print(f'   • Promoters avg checkout satisfaction: {promoters["checkout_satisfaction"].mean():.2f}/5')
print(f'   • Detractors avg checkout satisfaction: {detractors["checkout_satisfaction"].mean():.2f}/5')
print(f'   • Gap: {promoters["checkout_satisfaction"].mean() - detractors["checkout_satisfaction"].mean():.2f} points')

# Finding 5: Low performers
low_satisfaction = df[(df['checkout_satisfaction'] <= 2) | (df['search_satisfaction'] <= 2) | (df['returns_satisfaction'] <= 2)]
print(f'\n5. AT-RISK SEGMENT IDENTIFIED')
print(f'   • High friction users (any metric ≤2): {len(low_satisfaction)} customers ({len(low_satisfaction)/len(df)*100:.1f}%)')
print(f'   • These users avg NPS: {low_satisfaction["nps_score"].mean():.2f} (vs {df["nps_score"].mean():.2f} overall)')
print(f'   • Mobile users in this segment: {(low_satisfaction["channel_preference"]=='mobile').sum() / len(low_satisfaction) * 100:.1f}% (vs {(df["channel_preference"]=='mobile').sum() / len(df) * 100:.1f}% overall)')

# Finding 6: Tenure patterns
new_customers = df[df['tenure_cohort']=='New (0y)']
loyal_customers = df[df['tenure_cohort']=='Loyal (6y+)']
print(f'\n6. NEW CUSTOMERS NEED ONBOARDING INVESTMENT')
print(f'   • New customer avg NPS: {new_customers["nps_score"].mean():.2f}')
print(f'   • Loyal customer avg NPS: {loyal_customers["nps_score"].mean():.2f}')
print(f'   • Gap: {loyal_customers["nps_score"].mean() - new_customers["nps_score"].mean():.2f} points')
print(f'   • New customers in mobile channel: {(new_customers["channel_preference"]=='mobile').sum() / len(new_customers) * 100:.1f}%')
print(f'   • Opportunity: Mobile + onboarding → retention improvement')

print(f'\n' + '='*70)

In [ ]:
# Recommendations for product teams
print('\n' + '='*70)
print('RECOMMENDED ACTIONS'.center(70))
print('='*70)

print(f'''\n🎯 FOR CHECKOUT REDESIGN INITIATIVE (Alex's Team)\n
   1. MOBILE-FIRST APPROACH\n      • Prioritize mobile checkout improvements — {(df['channel_preference']=='mobile').sum()} users showing {desktop_checkout - mobile_checkout:.2f}pt satisfaction gap
      • Target: Lift mobile checkout satisfaction from {mobile_checkout:.2f} → 4.5+ /5
      • Quick wins: App crash fixes (mentioned by {len(df[df['improvement_suggestion'].str.contains('app crash|crash|timeout|clunky|slow', case=False, na=False)])} users)
   
   2. SEGMENT STRATEGY\n      • Phase 1: Non-member new users in mobile (highest friction + opportunity)
      • Phase 2: Bronze/Silver tier mobile users (volume opportunity)
      • Phase 3: Desktop experience enhancement (already strong at {desktop_checkout:.2f})
   
   3. SUCCESS METRICS\n      • Lift checkout satisfaction 0.5+ points → impact full NPS by ~{0.5 * corr_val:.2f} points
      • Reduce mobile-desktop gap from {desktop_checkout - mobile_checkout:.2f}pt to <0.3pt
      • Target: Mobile checkout detractors from {(df[(df['channel_preference']=='mobile')]['nps_score'] <= 6).sum()} → {int((df[(df['channel_preference']=='mobile')]['nps_score'] <= 6).sum() * 0.5)}

🏆 FOR LOYALTY PROGRAM V2 (Team Owner)\n
   1. TIER LEVERAGE\n      • Platinum members show {platinum_nps:.2f} NPS (vs {none_nps:.2f} for non-members)
      • Expand Silver tier benefits — highest upside (mid-tier growth)
      • Opportunity: Add exclusive checkout perks (express flow, free returns)
   
   2. TARGET EXPANSION\n      • Gold tier: {len(df[df['membership_tier']=='Gold'])} members, {df[df['membership_tier']=='Gold']['nps_score'].mean():.2f} NPS
      • Push Non-members → Bronze: {len(new_customers[new_customers['membership_tier']=='None'])} new customers without tier = high churn risk
      • Sustainability incentive: Multiple users mentioned eco-friendly products → loyalty bonus angle
   
   3. PROGRAM METRICS\n      • Current tier distribution suggests {len(df[df['membership_tier']=='None'])} customers (47%) unmonetized
      • Platinum members have {df[df['membership_tier']=='Platinum']['avg_order_value'].mean():.0f} avg order value
      • Convert 30% of non-members → Bronze = significant ARR impact

📊 RESEARCH PRIORITIES\n
   1. Qualitative interviews with {len(low_satisfaction)} at-risk customers (high friction)
   2. Mobile usability study focused on checkout flow (app crashes, timeouts)
   3. Non-member onboarding barriers (why {(df[df['membership_tier']=='None']['channel_preference']=='mobile').sum() / len(df[df['membership_tier']=='None']) * 100:.0f}% prefer mobile but have low satisfaction?)
   4. Returns process deep-dive (returns_satisfaction = {df['returns_satisfaction'].mean():.2f}, lowest metric)
''')

print('='*70)